In [ ]:
%load_ext autoreload
%autoreload 2




from fibsem import patterning
from fibsem import utils

from pprint import pprint

protocol  = utils.load_protocol("../protocol/protocol_serial.yaml")

pprint(protocol["horseshoe"])
stages = patterning._get_milling_stages("horseshoe", protocol)
ddict = patterning._get_protocol_from_stages(stages)
pprint(stages)
pprint(ddict)

### DETECTION TESTS

In [ ]:
%load_ext autoreload
%autoreload 2


from fibsem.detection import detection
from fibsem import utils
import tifffile as tff

import numpy as np
import matplotlib.pyplot as plt


import glob
import os

from fibsem.segmentation.utils import decode_segmap

PATH = "../../../fibsem/fibsem/detection/test_images/serial"
filenames = sorted(glob.glob(os.path.join(PATH,  "*.tif")))

print(filenames)

# fig , ax = plt.subplots(1, len(filenames), figsize=(40, 15))

features = [
    [detection.NeedleTipBottom()], # prep0
    [detection.NeedleTipBottom(), detection.LandingPost()],
    [detection.LamellaTopEdge()],
    [detection.NeedleTipBottom(), detection.CopperAdapterBottomEdge(),  detection.LamellaTopEdge(), detection.LamellaBottomEdge()],
    [detection.NeedleTipBottom(), detection.LamellaTopEdge(), detection.LamellaBottomEdge()],
    [detection.LamellaBottomEdge(), detection.LandingPost()],
    [detection.LamellaTopEdge(), detection.LamellaBottomEdge(), detection.LandingPost()],
    [ detection.LamellaTopEdge(), detection.LamellaBottomEdge()],

]

feature = detection.NeedleTipBottom()





In [ ]:

from fibsem.structures import Point

for i, (fname, flist) in enumerate(zip(filenames, features)):
    mask = tff.imread(fname)

    for feature in flist:
        feature.detect(img=mask, mask=mask, point=Point(mask.shape[1]//2, mask.shape[0]//2))


    plt.imshow(mask)
    plt.imshow(decode_segmap(mask, nc=4))
    plt.title(os.path.basename(fname).removesuffix(".tif"))
    for feature in flist:
        plt.plot(feature.px.x, feature.px.y, color=feature.color,
            markeredgecolor="w", 
            marker="o", markersize=5, label=feature.name)
    plt.legend(loc="best")

    plt.show()


### Landing Stage

In [ ]:
%load_ext autoreload
%autoreload 2


from fibsem import utils, acquire, movement
from fibsem.imaging import _tile

from fibsem.structures import BeamType, Point, FibsemStagePosition

import numpy as np
import matplotlib.pyplot as plt

from pprint import pprint 

# connect
microscope, settings = utils.setup_session(protocol_path="../protocol/protocol_serial.yaml")

# pprint(settings.protocol)


In [ ]:
from fibsem import patterning
from fibsem.structures import Point




stages = patterning._get_milling_stages("weld", settings.protocol)
pprint(stages)


stages = patterning._get_milling_stages("weld", settings.protocol, [Point(0.5, 0.5), Point(-0.5, -0.5)])

pprint(stages)


stages = patterning._get_milling_stages("liftout_weld", settings.protocol)

pprint(stages)

stages = patterning._get_milling_stages("liftout_weld", settings.protocol, Point(0.5, 0.5))

pprint(stages)

In [ ]:

from liftout.workflows import serial



In [ ]:
settings.image.resolution = [1024, 1024]
settings.image.beam_type = BeamType.ION
image = _tile._tile_image_collection_stitch(microscope, settings, 2000e-6, 500e-6, )


In [ ]:
# plot the positions
fig = _tile._plot_positions(image, positions)

plt.show()

# TODO generate these onto a minimap image



In [ ]:

from liftout.structures import Lamella, Experiment, AutoLiftoutState, AutoLiftoutStage

import os
from pathlib import Path

from copy import deepcopy

PATH = "exp/test/"
os.makedirs(PATH, exist_ok=True)

# experiment = Experiment(PATH, name="DEV-SERIAL-TEST-01", method="serial-liftout")

# if lamella is at Liftout: 
#   continue
# else:
#  return

# calculate landing positions
# get index of lamella (num that have landed)
# create lamella




In [ ]:
from autolamella.tools.data import create_history_dataframe

display(experiment.to_dataframe_v2())

df = create_history_dataframe(experiment)
display(df)

COLOURS = ["lime", "blue", "cyan", "magenta", 
    "hotpink", "yellow", "orange", "red"]

# plot the positions

for i, lamella in enumerate(experiment.positions):
    c =COLOURS[i%len(COLOURS)]
    pos = lamella.state.microscope_state.absolute_position 
    plt.plot(pos.x, pos.y, c=c, marker='+', label=lamella._petname)
    # text
    # plt.text(pos.x-100, pos.y-50, lamella._petname, fontsize=12, color=c, alpha=0.75)
    plt.text(pos.x-25e-6, pos.y+50e-6, lamella._petname, fontsize=8, color=c, alpha=0.75)
# plt.legend(loc='best')

plt.xlabel('x [µm]')
plt.ylabel('y [µm]')
plt.title('Lamella positions')
plt.show()




In [ ]:
features = [detection.LamellaBottomEdge(), detection.LandingPost()]

det = detection.take_image_and_detect_features(
    microscope=microscope,
    settings=settings,
    features=features,
)

In [ ]:
pprint(det)